<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/shieh2020_survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [3]:
!pip install scipy --upgrade

In [2]:
from scipy.stats import rankdata


paper : https://doi.org/10.1371/journal.pone.0233188



 " IDH and MGMT markers were not mandatory for routine pathologic reports " Sadly we won't have this markers in this one, but we have it in other datasets.

In [4]:
dataset = pd.read_excel('/content/shieh.xlsx')

In [5]:
dataset.head(5)

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1,63,1,1,1,6000,1.0,1,1,2,1.36,1.260274,731.0,1,63
1,1,64,1,1,0,6600,0.0,2,1,1,8.84,5.115068,NaN,1,64
2,1,62,1,1,1,6000,1.0,1,1,2,1.46,0.641096,215.0,1,62
3,1,71,1,1,1,6000,NaN,3,1,1,2.42,0.652055,255.0,1,71
4,1,21,1,1,0,5940,0.0,2,1,2,1.26,1.079452,154.0,1,21


In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Let's do all the pre-processing needed

In [7]:
imp_mean = IterativeImputer(random_state=0)

In [8]:
data = imp_mean.fit_transform(dataset)

In [9]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [10]:
data.head()

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
0,1.0,63.0,1.0,1.0,1.0,6000.0,1.000000,1.0,1.0,2.0,1.36,1.260274,731.000000,1.0,63.0
1,1.0,64.0,1.0,1.0,0.0,6600.0,0.000000,2.0,1.0,1.0,8.84,5.115068,272.938289,1.0,64.0
2,1.0,62.0,1.0,1.0,1.0,6000.0,1.000000,1.0,1.0,2.0,1.46,0.641096,215.000000,1.0,62.0
3,1.0,71.0,1.0,1.0,1.0,6000.0,-0.999998,3.0,1.0,1.0,2.42,0.652055,255.000000,1.0,71.0
4,1.0,21.0,1.0,1.0,0.0,5940.0,0.000000,2.0,1.0,2.0,1.26,1.079452,154.000000,1.0,21.0


In [11]:
data.corr(method = 'pearson')

,gender,age,statusPFS,statusOS,surgery,dose,TMZ,Chemogroup,Chemo,diagnostic year,OS,PFS,volume,gender.1,age.1
gender,1.000000,-0.163487,-0.139923,-0.113228,0.169842,0.117790,0.159429,-0.153072,-0.030024,-0.037493,0.043419,0.080638,-0.073217,1.000000,-0.163487
age,-0.163487,1.000000,0.202956,0.075994,-0.005687,-0.199397,0.046168,0.037592,0.110030,0.089632,-0.188313,-0.254452,0.153293,-0.163487,1.000000
statusPFS,-0.139923,0.202956,1.000000,0.570352,-0.063372,0.014962,-0.091558,0.045091,-0.044811,-0.075545,-0.022415,-0.143078,0.011984,-0.139923,0.202956
statusOS,-0.113228,0.075994,0.570352,1.000000,-0.111111,0.026233,-0.160530,0.079058,-0.078567,-0.132453,-0.028040,-0.045293,-0.083594,-0.113228,0.075994
surgery,0.169842,-0.005687,-0.063372,-0.111111,1.000000,-0.033926,0.171413,0.020329,0.235702,0.132453,-0.118900,-0.001697,0.088748,0.169842,-0.005687
dose,0.117790,-0.199397,0.014962,0.026233,-0.033926,1.000000,-0.132206,0.315258,0.297829,-0.334150,0.468474,0.378734,-0.162929,0.117790,-0.199397
TMZ,0.159429,0.046168,-0.091558,-0.160530,0.171413,-0.132206,1.000000,-0.730626,0.144294,0.515709,0.092544,0.077334,-0.057429,0.159429,0.046168
Chemogroup,-0.153072,0.037592,0.045091,0.079058,0.020329,0.315258,-0.730626,1.000000,0.570208,-0.370692,0.002759,-0.051486,0.013387,-0.153072,0.037592
Chemo,-0.030024,0.110030,-0.044811,-0.078567,0.235702,0.297829,0.144294,0.570208,1.000000,0.083252,0.115345,0.018430,-0.049697,-0.030024,0.110030
diagnostic year,-0.037493,0.089632,-0.075545,-0.132453,0.132453,-0.334150,0.515709,-0.370692,0.083252,1.000000,-0.345445,-0.203701,-0.077207,-0.037493,0.089632


In [12]:
data.drop(columns = ['TMZ', 'gender.1', 'age.1',], axis = 1, inplace= True)

In [13]:
data.drop(columns = ['Chemogroup', 'diagnostic year'], axis = 1, inplace = True)

Converting Y to the ndarray so the sksurv package can process it

In [15]:
Y = data[['statusOS', 'OS']]

In [16]:
X = data.drop(columns=['statusOS', 'OS'])

In [ ]:
Y['statusOS'] = Y['statusOS'].map({1.0: True, 0: False})


In [18]:
Y = Y.to_records(index=False)

In [19]:
Y

rec.array([( True, 1.36), ( True, 8.84), ( True, 1.46), ( True, 2.42),
           ( True, 1.26), ( True, 0.85), ( True, 6.43), ( True, 0.63),
           ( True, 0.96), ( True, 1.32), ( True, 2.82), ( True, 0.77),
           ( True, 0.84), (False, 2.38), ( True, 0.23), ( True, 1.  ),
           ( True, 3.5 ), ( True, 1.18), ( True, 5.4 ), ( True, 1.93),
           ( True, 0.59), ( True, 0.48), ( True, 2.25), ( True, 1.34),
           ( True, 0.9 ), ( True, 1.12), ( True, 0.58), ( True, 1.01),
           ( True, 0.88), ( True, 0.62), ( True, 1.59), ( True, 2.33),
           ( True, 1.66), ( True, 1.11), ( True, 0.79), ( True, 1.11),
           ( True, 2.92), ( True, 1.22), ( True, 1.9 ), ( True, 2.5 ),
           ( True, 0.32), ( True, 2.22), ( True, 9.15), ( True, 0.45),
           ( True, 1.37), ( True, 3.45), ( True, 5.07), ( True, 1.82),
           ( True, 0.41), ( True, 0.95), ( True, 0.98), ( True, 1.38),
           ( True, 1.82), ( True, 2.68), ( True, 0.89), ( True, 1.68),
      

# Let's do the ranking

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

In [21]:
pip install eli5

     |████████████████████████████████| 106 kB 7.7 MB/s 


In [22]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from eli5.sklearn import PermutationImportance
import eli5


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [23]:
rsf = RandomSurvivalForest()


In [24]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  bordarank = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)

  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test set
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X_train, y_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much the Concordance score dropped
    feature_importance = permuter.feature_importances_
  # I need to create a ranking out of this
    rankings = np.add(feature_importance, rankings)
  #This will create a numpy array with each c index loss. Each row is a loop. 
    bordarank = np.vstack((bordarank, feature_importance))
  #Let's compute the Concordance index score
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.DataFrame({'features': columns, 'C_loss': ranking_terminal}).sort_values(ascending = False, by =['C_loss'])
  c_mean = np.mean(np.array(c_score_total))
  c_std = np.std(np.array(c_score_total))
  #Let's start working with the borda. First we delete the first row that is just 0s
  bordarank = np.delete(bordarank, 0, 0)
  ranking_the_data = rankdata(bordarank * -1, axis=1)
  rankavg = np.mean(ranking_the_data, axis = 0)
  ranksd = np.std(ranking_the_data, axis = 0)
  #Vamos a ver como queda
  borda = pd.DataFrame({'Features': columns, 'C_Average_Loss': ranking_terminal, 'borda_average': rankavg, 'borda_sd': ranksd}).sort_values(ascending = True, by = ['borda_average'])
  



  return c_rank, c_mean, c_std, borda

In [25]:
rank, c_mean, c_std, borda = permutation(X, Y, rsf, 100)

In [26]:
rank

,features,C_loss
6,PFS,0.192140
1,age,0.015872
4,dose,0.008050
5,Chemo,0.002781
7,volume,0.001113
2,statusPFS,0.000000
0,gender,-0.001096
3,surgery,-0.001656


In [27]:
c_mean

0.7440020681459888

In [28]:
c_std

0.04397488082952339

In [29]:
borda

,Features,C_Average_Loss,borda_average,borda_sd
6,PFS,0.192140,1.000,0.000000
1,age,0.015872,3.750,2.137171
4,dose,0.008050,4.290,1.865717
5,Chemo,0.002781,4.975,1.605265
7,volume,0.001113,5.085,2.391710
2,statusPFS,0.000000,5.480,1.421126
0,gender,-0.001096,5.595,1.740251
3,surgery,-0.001656,5.825,1.778166


Now let's try the first 2 feature.

In [ ]:
X_tf = data[['PFS', 'age']]

In [ ]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  c_std_top2 = np.std(np.array(c_score_total))



  return c_mean_top2

In [ ]:
c_score_tf = top2features(X_tf, Y, rsf, 100)

In [ ]:
c_score_tf 

0.7707494191374952

## Brier Score


In [30]:
pip install pysurvival

     |████████████████████████████████| 4.7 MB 8.1 MB/s 
  Created wheel for pysurvival: filename=pysurvival-0.1.2-cp37-cp37m-linux_x86_64.whl size=3773235 sha256=424688a2c80ca8107112b12cbf3f31574d1618b3e8150c998f746aeadaca9e09
  Stored in directory: /root/.cache/pip/wheels/1a/63/e2/32273d765a4e2f4ccac69c8adf97425ca80bab5d0c8447f120
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=38084d00d8771ac54794e087ade4cd46c0e3b06b130ae2ba33e467077aadbbb1
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built pysurvival progressbar


In [31]:
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.metrics import integrated_brier_score
from pysurvival.models.survival_forest import RandomSurvivalForestModel

In [32]:
T = data['OS']
E = data['statusOS']

In [33]:
pyforest = RandomSurvivalForestModel(num_trees=100)

In [34]:
def Py_scores(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, max_depth=5)
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))


  return ibstotal2, ibs_std

In [35]:
ibs_mean, ibs_std = Py_scores(X, T, E, pyforest, 100)

/usr/local/lib/python3.7/dist-packages/pysurvival/models/survival_forest.py:399: RuntimeWarning: invalid value encountered in true_divide
  self.variable_importance_table['pct_importance']= importance/sum_imp


In [36]:
ibs_mean

0.13725333133228546

In [37]:
ibs_std

0.024911599066136287

## Deep Surv

In [38]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel


In [39]:
def Py_scores_deepsurv(X, T, E, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the NN 
    structure = [ {'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 64}, ]
    nonlinear_coxph = NonLinearCoxPHModel(structure = structure)
    nonlinear_coxph.fit(X_train, t_train, e_train, num_epochs = 100, dropout = 0.2,)
    c_score_total.append(concordance_index(nonlinear_coxph, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(nonlinear_coxph, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_score_avg = np.mean(np.array(c_score_total))
  c_score_std = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_score_avg, c_score_std

In [ ]:
ib, ibstd, c_index_avg, c_score_std = Py_scores_deepsurv(X, T, E, 100)

In [41]:
c_index_avg

0.6897279533286147

In [42]:
c_score_std

0.06999610522329555

In [43]:
ib

0.1197420629534416

In [44]:
ibstd

0.02567779284053625

## Cox Proportional Hazards


In [45]:
from pysurvival.models.semi_parametric import CoxPHModel


In [46]:
coxph = CoxPHModel()


In [47]:
def Py_scores_cox(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train)
    c_score_total.append(concordance_index(model, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_index = np.mean(np.array(c_score_total))
  c_index_sd = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_index, c_index_sd

In [ ]:
ib, ibstd, c_index_avg, c_index_sd = Py_scores_cox(X, T, E, coxph, 100)

In [49]:
ib

0.11500412911909985

In [50]:
ibstd

0.02680322751157355

In [51]:
c_index_avg

0.6799917848326246

In [52]:
c_index_sd

0.07589780599720908